In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
main_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
main_df = pd.merge(main_df,items,on='item_id')
main_df = main_df.drop(['item_id','item_name'],axis=1)
main_df_grouped = main_df.groupby(['date_block_num','item_category_id','shop_id'])["item_price","item_cnt_day"].agg({"item_price":"mean","item_cnt_day":"sum"}).reset_index()

In [ ]:
def get_prev_sales(shop_id,item_category_id,date_block_num,lag,df):
    df = df[(df['shop_id']==shop_id)&(df['item_category_id']==item_category_id)&(df['date_block_num']==date_block_num-lag)]
    #df = main_df.query('shop_id == @shop_id and item_id == @item_id and date_block_num==@date_block_num-@lag')
    if len(df)==0:
        return np.nan
    else:
        #print(len(df))
        return np.sum(df['item_cnt_day'])

In [ ]:
main_df_grouped['month-1'] = main_df_grouped.apply(lambda row:get_prev_sales(row['shop_id'],row['item_category_id'],row['date_block_num'],1,df=main_df_grouped),axis=1)

In [ ]:
for i in range(1,6):
    main_df_grouped['month-'+str(i)] = main_df_grouped.apply(lambda row:get_prev_sales(row['shop_id'],row['item_category_id'],row['date_block_num'],i,df=main_df_grouped),axis=1)

In [ ]:
main_df_grouped.to_csv('prepared_data.csv')

In [ ]:
main_df_grouped.dropna(subset=['month-1','month-2','month-3','month-4','month-5'],how='any')

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

In [ ]:
my_model = Sequential()
my_model.add(LSTM(units = 1,input_shape = (5,1)))
#my_model.add(Dropout(0.4))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

# Colab Content Start

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# Adapted from https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu

PATH_TPU_WORKER = ''

def check_tpu():
    """
    Detect TPU hardware and return the appopriate distribution strategy
    """
    
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
        print('Running on TPU: {}'.format(tpu.master()))
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        tpu_strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

    print("Num. replicas: {}".format(tpu_strategy.num_replicas_in_sync))
    
    return tpu, tpu_strategy
    
tpu, tpu_strategy = check_tpu()
PATH_TPU_WORKER = tpu.master()
NUM_REPLICAS = tpu_strategy.num_replicas_in_sync

In [ ]:
data = pd.read_csv('../input/prepared-data/prepared_data.csv',engine='python')
data = data.dropna(subset=['month-1','month-2','month-3','month-4','month-5'],how='any')
data = data.drop(columns=['Unnamed: 0','date_block_num','item_price','item_category_id','shop_id'])

In [ ]:
dataset = data.values
dataset = dataset.astype('float32')
datasetX, datasetY = dataset[:,[1,2,3,4,5]],dataset[:,0]
# normalize the dataset
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
datasetX = x_scaler.fit_transform(datasetX)
datasetY = y_scaler.fit_transform(datasetY.reshape(-1, 1))
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
trainX, testX = datasetX[0:train_size,:], datasetX[train_size:len(datasetX),:]
trainY, testY = datasetY[0:train_size,:], datasetY[train_size:len(datasetY),:]

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, 5)))
#model.add(Dropout(0.4))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


In [ ]:
# invert predictions
trainPredict = y_scaler.inverse_transform(trainPredict)
trainY = y_scaler.inverse_transform(trainY)
testPredict = y_scaler.inverse_transform(testPredict)
testY = y_scaler.inverse_transform(testY)

In [ ]:
np.sqrt(((trainPredict-trainY)**2).sum())

# Colab Content End